In [7]:
import json,os,sys,logging,pprint,re,string,courseraLib,random

In [20]:
sample_rate=10 #1/1000
path='/home/yewenhe0904/Developing/ds-capstone-project/testing-data/mixed-testing.txt'
testing_set=open(path)


# Initial call to print 0% progress
i = 0
l = len(list(open(path)))
l

Testset=[[], [], [], [], [], [], [], [], [], []]
Testset

courseraLib.printProgressBar(i, l, prefix = 'TestSet'+":", suffix = 'Complete')
for line in testing_set:
    p=random.randint(1,1000)
    if (p)<=sample_rate:
#         print('\n',p)
        EoSentence="[.,;!?]+"
        line_test=re.sub(EoSentence,"\n",line)
        line_test=line_test.split("\n")
        line_in=line_test[0]
        if len(line_in)<5:
            continue
        if len(line_in)>15:
            line_in=line_in[random.randint(5,10)]
        Testset[(p-1)].append(line_test[0])
    i+=1
    courseraLib.printProgressBar(i, l, prefix = 'TestSet'+":", suffix = 'Complete')

In [29]:
nGramDict=\
json.load(open('/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/Sample-Dict/mixed-nGramDict_50_5.json'))

def cleanInputPredict(test_input):
    cleanedInput=[]
    test_tokens=test_input.strip().lower().split()
    test_input=re.sub('\n+'," ",test_input)
    for item in test_tokens:
        item = item.strip(string.punctuation)
        item = re.sub(r"^.*\d+.*$","<Quantity>",item)
        cleanedInput.append(item)
    return cleanedInput

def predictCore(input_tokens,output_list,out_len=5):
#     print (input_tokens)
    if len(input_tokens)<1:
        output_list+=(nGramDict['_r'])
    else:
        # probe if in ngram record
        nGram_flag=True
        i=0
        p_Dict=nGramDict['_n']
        while nGram_flag:
#             print(i)
            if i==len(input_tokens):
                break
            if (input_tokens[i] in p_Dict):
                r_wids=p_Dict[input_tokens[i]]['_r']
                if ('_n' in p_Dict[input_tokens[i]]):
                    p_Dict=p_Dict[input_tokens[i]]['_n']
                    i+=1
                else:
                    if i<len(input_tokens)-1:
                        nGram_flag=False
                    break
            else:
                nGram_flag=False
                break
        # if true
        if nGram_flag:
            output_list+=r_wids
            if len(output_list)<out_len:
                del input_tokens[0]
                predictCore(input_tokens,output_list,out_len=out_len)
        # else false
        else:
            del input_tokens[0]
            predictCore(input_tokens,output_list,out_len=out_len)

def predictNgram(test_input,out_len):
    #clean input string
    input_tokens=cleanInputPredict(test_input)
    if len(input_tokens)>nGramDict['_model']:
        input_tokens=input_tokens[-nGramDict['_model']:]
    output_list=[]
    output_wlist=[]
    output=[]
    input_idtokens=[]
    for w in input_tokens:
        w.strip()
        if w in nGramDict['_word2id']['_word']:
            input_idtokens.append(nGramDict['_word2id']['_word'][w])
        else:
            input_idtokens.append('NA')
    predictCore(input_idtokens,output_list,out_len)
    for wid in output_list:
        output_wlist.append(nGramDict['_id2word']['_id'][wid])
    eosflag=True
#     print(output_wlist)
    for word in output_wlist:
        if word=='<EOS>':
            if eosflag:
                output.append(',')
                output.append('.')
            eosflag=False
        elif word=="<Quantity>":
            continue
        else:
            output.append(word)
        #dedupe
        output_dp=[output[0]]
        if len(output)>1:
            for i in range(1,len(output)):
                if output[i] not in output[:i]:
                    output_dp.append(output[i])
        if len(output_dp)>out_len:
            output_dp=output_dp[0:out_len]
    return output_dp

In [30]:
i=0
total_t=0
right_t=0
for t in Testset:
    i+=1
    print('Test',i,":")
    total=0
    right=0
    print('length of current test:',len(t))
    for s in t:
        stest=s.strip().lower().split()
        key=stest[-1]
        del stest[-1]
        q_input=' '.join(stest)
        if key in predictNgram(q_input,10):
            right+=1
        total+=1
    print(right,'/',total,str(round(right/total*100,2))+'%')
    total_t+=total
    right_t+=right
print('TOTAL AVERAGE: ',right_t,'/',total_t,str(round(right_t/total_t*100,2))+'%')

    

Test 1 :
length of current test: 153
47 / 153 30.72%
Test 2 :
length of current test: 162
48 / 162 29.63%
Test 3 :
length of current test: 162
47 / 162 29.01%
Test 4 :
length of current test: 151
41 / 151 27.15%
Test 5 :
length of current test: 182
46 / 182 25.27%
Test 6 :
length of current test: 156
36 / 156 23.08%
Test 7 :
length of current test: 163
42 / 163 25.77%
Test 8 :
length of current test: 177
50 / 177 28.25%
Test 9 :
length of current test: 158
40 / 158 25.32%
Test 10 :
length of current test: 160
45 / 160 28.12%
TOTAL AVERAGE:  442 / 1624 27.22%
